# Data Science in the Cloud: The "Azure ML SDK" way 

## Introduction

In this notebook, we will learn how to use the Azure ML SDK to train, deploy and consume a model through Azure ML.

Pre-requisites:
1. You created an Azure ML workspace.
2. You loaded the [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) into Azure ML.
3. You uploaded this notebook into Azure ML Studio.

The next steps are:

1. Create an Experiment in an existing Workspace.
2. Create a Compute cluster.
3. Load the dataset.
4. Configure AutoML using AutoMLConfig.
5. Run the AutoML experiment.
6. Explore the results and get the best model.
7. Register the best model.
8. Deploy the best model.
9. Consume the endpoint.

## Azure Machine Learning SDK-specific imports

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

nahmed30-azureml-workspace
epe-poc-nazeer
centralus
16bc73b5-82be-47f2-b5ab-f2373344794c


## Create an Azure ML experiment

Let's create an experiment named 'aml-experiment' in the workspace we just initialized.

In [3]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
aml-experiment,nahmed30-azureml-workspace,Link to Azure Machine Learning studio,Link to Documentation


## Create a Compute Cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) for your AutoML run.

In [4]:
aml_name = "cpu-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

Found existing AML compute context.


## Data
Make sure you have uploaded the dataset to Azure ML and that the key is the same name as the dataset.

In [6]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML Run

In [9]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
aml-experiment,AutoML_34e53b7b-518f-4446-9c00-27a0fa012344,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Save the best model

In [11]:
best_run, fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.44.0, current version:1.41.0
Package:azureml-core, training version:1.44.0, current version:1.41.0
Package:azureml-dataprep, training version:4.2.2, current version:3.1.1
Package:azureml-dataprep-rslex, training version:2.8.1, current version:2.5.2
Package:azureml-dataset-runtime, training version:1.44.0, current version:1.41.0
Package:azureml-defaults, training version:1.44.0, current version:1.41.0
Package:azureml-inference-server-http, training version:0.7.4, current version:0.4.13
Package:azureml-interpret, training version:1.44.0, current version:1.41.0
Package:azureml-mlflow, training version:1.44.0, current version:1.41.0
Package:azureml-pipeline-core, training version:1.44.0, current version:1.41.0
Package:azureml-responsibleai, training version:1.44.0, current version:1.41.0
Package:azureml-telemetry, training version:1.44.0, current version:1.41.0
Package:azureml-train-automl-client, training version:1.44.0, current version:1

In [12]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'aml-experiment\',\'compute_target\':\'cpu-cluster\',\'subscription_id\':\'16bc73b5-82be-47f2-b5ab-f2373344794c\',\'region\':\'centralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_34e53b7b-518f-4446-9c00-27a0fa012344_34","experiment_name":"aml-experiment","workspace_name":"nahmed30-azureml-workspace","subscription_id":"16bc73b5-82be-47f2-b5ab-f2373344794c","resource_group_name":"epe-poc-nazeer"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '34',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.ComputeTargetType': 

In [16]:
for child_run in remote_run.get_children():
    print(child_run,"\n********************\n")

Run(Experiment: aml-experiment,
Id: AutoML_34e53b7b-518f-4446-9c00-27a0fa012344_35,
Type: azureml.scriptrun,
Status: Completed) 
********************

Run(Experiment: aml-experiment,
Id: AutoML_34e53b7b-518f-4446-9c00-27a0fa012344_34,
Type: azureml.scriptrun,
Status: Completed) 
********************

Run(Experiment: aml-experiment,
Id: AutoML_34e53b7b-518f-4446-9c00-27a0fa012344_33,
Type: azureml.scriptrun,
Status: Canceled) 
********************

Run(Experiment: aml-experiment,
Id: AutoML_34e53b7b-518f-4446-9c00-27a0fa012344_32,
Type: azureml.scriptrun,
Status: Canceled) 
********************

Run(Experiment: aml-experiment,
Id: AutoML_34e53b7b-518f-4446-9c00-27a0fa012344_31,
Type: azureml.scriptrun,
Status: Completed) 
********************

Run(Experiment: aml-experiment,
Id: AutoML_34e53b7b-518f-4446-9c00-27a0fa012344_30,
Type: azureml.scriptrun,
Status: Completed) 
********************

Run(Experiment: aml-experiment,
Id: AutoML_34e53b7b-518f-4446-9c00-27a0fa012344_29,
Type: azurem

In [14]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"


In [15]:
model_name

'AutoML34e53b7b534'

In [17]:
os.makedirs('./outputs',exist_ok=True)

In [18]:
#TODO: Save the best model
import joblib
joblib.dump(fitted_model,filename= "outputs/automl.joblib")

['outputs/automl.joblib']

In [19]:

from azureml.core.model import Model
reg_model = remote_run.register_model(model_name = model_name, description=description)


In [20]:
from azureml.automl.core.shared import constants
env = best_run.get_environment()
script_file = "./outputs/score.py"

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'env.yml')

## Deploy the Best Model

Run the following code to deploy the best model. You can see the state of the deployment in the Azure ML portal. This step can take a few minutes.

In [21]:
script_file_name

'inference/score.py'

In [22]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [reg_model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-09-02 05:57:54+00:00 Creating Container Registry if not exists.
2022-09-02 05:57:54+00:00 Registering the environment.
2022-09-02 05:57:55+00:00 Use the existing image.
2022-09-02 05:57:55+00:00 Submitting deployment to compute.
2022-09-02 05:57:59+00:00 Checking the status of deployment automl-hf-sdk..
2022-09-02 06:00:22+00:00 Checking the status of inference endpoint automl-hf-sdk.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## Consume the Endpoint
You can add inputs to the following input sample. 

In [27]:
scoring_uri = aci_service.scoring_uri
print(scoring_uri)

http://d4994745-199b-4acb-bb79-77448fbbb00b.centralus.azurecontainer.io/score


In [29]:
import requests
import json

 
data = {
  "data": [
    {
      "age": 0.0,
      "anaemia": 0,
      "creatinine_phosphokinase": 0,
      "diabetes": 0,
      "ejection_fraction": 0,
      "high_blood_pressure": 0,
      "platelets": 0.0,
      "serum_creatinine": 0.0,
      "serum_sodium": 0,
      "sex": 0,
      "smoking": 0,
      "time": 0
    }
  ],
  "method": "predict"
}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("prediction is :" , resp.json())

prediction is : {"result": [1]}
